<a href="https://colab.research.google.com/github/iamjems/Enhancement-of-Surveillance-Cameras-Accident-Detection-with-Alert-System/blob/main/Partial_Accident_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mounting Files on Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd '/content/drive/MyDrive/Colab Notebooks/test folder'

In [ ]:
%pwd

In [ ]:
!git  clone https://github.com/iamjems/Enhancement-of-Surveillance-Cameras-Accident-Detection-with-Alert-System.git



# Imports

In [ ]:
from ctypes import *
import math
import random
import os
import cv2
import numpy as np
import time
import darknet
from itertools import combinations
import pafy
import youtube_dl
#import image_email_car   # Uncomment for alert to email


# Calculate Euclidean Distance between two points

In [1]:
def is_close(p1, p2):
    """    
    :param:
    p1, p2 = two points for calculating Euclidean Distance
    :return:
    dst = Euclidean Distance between two 2d points
    """
    dst = math.sqrt(p1**2 + p2**2)
    return dst 



# Center coordinates to rectangle coordinates Conversion


In [2]:
def convertBack(x, y, w, h): 
    
    """
    :param:
    x, y = midpoint of bbox
    w, h = width, height of the bbox
    
    :return:
    xmin, ymin, xmax, ymax
    """
    xmin = int(round(x - (w / 2)))
    xmax = int(round(x + (w / 2)))
    ymin = int(round(y - (h / 2)))
    ymax = int(round(y + (h / 2)))
    return xmin, ymin, xmax, ymax

#alert_var = 0      # Uncomment for alert to email       # makes sure that alert (Sending an E-mail) is generated only once

# cvDrawBoxes

In [ ]:
def cvDrawBoxes(detections, img):
    """
    :param:
    detections = total detections in one frame
    img = image from detect_image method of darknet
    :return:
    img with bbox
    """
    #global alert_var

Filter out Cars class from detections and get bounding box centroid for each car detection.

In [ ]:
 if len(detections) > 0:  						# At least 1 detection in the image and check detection presence in a frame  
        centroid_dict = dict() 						# Function creates a dictionary and calls it centroid_dict
        objectId = 0								# We inialize a variable called ObjectId and set it to 0
        for detection in detections:				# In this if statement, we filter all the detections for cars only
            # Check for the only car name tag 
            name_tag = str(detection[0].decode())   # Coco file has string of all the names
            if name_tag == 'car':                
                x, y, w, h = detection[2][0],\
                            detection[2][1],\
                            detection[2][2],\
                            detection[2][3]      	# Store the center points of the detections
                xmin, ymin, xmax, ymax = convertBack(float(x), float(y), float(w), float(h))   # Convert from center coordinates to rectangular coordinates, We use floats to ensure the precision of the BBox            
                # Append center point of bbox for cars detected.
                centroid_dict[objectId] = (int(x), int(y), xmin, ymin, xmax, ymax) # Create dictionary of tuple with 'objectId' as the index center points and bbox
                objectId += 1 #Increment the index for each detection    

Determine which car bbox are close to each other

In [ ]:
 vehicle_red_zone_list = [] # List containing which Object id is in under threshold distance condition. 
        vehicle_red_line_list = []
        for (id1, p1), (id2, p2) in combinations(centroid_dict.items(), 2): # Get all the combinations of close detections, #List of multiple items - id1 1, points 2, 1,3
            #dx, dy = p1[0] - p2[0], p1[1] - p2[1]  	# Check the difference between centroid x: 0, y :1
            #distance = is_close(dx, dy) 			# Calculates the Euclidean distance
            
            #if distance < 50.0:						# Set our distance threshold - If they meet this condition then..
            
            if not ((p1[2]>=p2[4]) or (p1[4]<=p2[2]) or (p1[5]<=p2[3]) or (p1[3]>=p2[5])):
                if id1 not in vehicle_red_zone_list:
                    vehicle_red_zone_list.append(id1)       #  Add Id to a list
                    vehicle_red_line_list.append(p1[0:2])   #  Add points to the list
                if id2 not in vehicle_red_zone_list:
                    vehicle_red_zone_list.append(id2)		# Same for the second id 
                    vehicle_red_line_list.append(p2[0:2])
        
        for idx, box in centroid_dict.items():  # dict (1(key):red(value), 2 blue)  idx - key  box - value
            if idx in vehicle_red_zone_list:   # if id is in red zone list
                cv2.rectangle(img, (box[2], box[3]), (box[4], box[5]), (255, 0, 0), 2) # Create Red bounding boxes  #starting point, ending point size of 2
            else:
                cv2.rectangle(img, (box[2], box[3]), (box[4], box[5]), (0, 255, 0), 2) # Create Green bounding boxes